Create df of a team's schedule with stadium played in, attendance, and capacity

In [11]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import json

In [3]:
start_date = 20221107
womens = True
schedule_url = f"https://www.espn.com/womens-college-basketball/schedule/_/date/"
stadiums_url = "https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_basketball_arenas"

Create table of stadiums

In [4]:
stadiums_table = pd.read_html(stadiums_url)

In [5]:
stadiums_df = stadiums_table[1].drop('Image',axis=1)
stadiums_df.head()

,Arena,City,State,Team,Conference,Capacity,Opened
0,Conte Forum,Chestnut Hill,MA,Boston College,ACC,8606,1988
1,Littlejohn Coliseum,Clemson,SC,Clemson,ACC,9000,1968
2,Cameron Indoor Stadium,Durham,NC,Duke,ACC,9314,1940
3,Donald L. Tucker Civic Center,Tallahassee,FL,Florida State,ACC,12100,1981
4,Hank McCamish Pavilion,Atlanta,GA,Georgia Tech,ACC,8600,1956


In [6]:
schedule_page = rq.get(schedule_url + str(start_date))

In [7]:
schedule_soup = bs(schedule_page.content)
schedule_table = schedule_soup.table

In [8]:
data = {'Home_Team': [],
        'Away_Team': [],
        'Attendance': [],
        'Capacity': [],
        'Percent_Capacity_Filled' : [],
        'Date': [],
        'State':[],
        'City': [],
        'Arena': []} 

date = schedule_soup.find('div', {'class': 'Table__Title'}).contents[0]
for row in schedule_table.tbody:
    row_cells = row.find_all('td')
    try:
        home_team = row_cells[0].find_all('a')[1].contents[0]
        away_team = row_cells[1].find_all('a')[1].contents[0]
        game_link = row_cells[2].find('a')['href']
    except:
        # break
        pass
    
    # find attendance number
    game_page = rq.get(
        "http://espn.com"+ game_link)
    game_soup = bs(game_page.content)
    atndnce = game_soup.find('div', {"class":"Attendance__Numbers"})
    atndnce = atndnce.contents[4] if atndnce != None else 0
    atndnce = ''.join(filter(str.isdigit, atndnce)) if atndnce != 0 else atndnce
    
    capacity = game_soup.find('div', {"class": "Attendance__Capacity"})
    capacity = capacity.contents[4] if capacity != None else 0
    capacity = ''.join(filter(str.isdigit, capacity)
                        ) if capacity != 0 else capacity
    
    percent_cap = round(int(atndnce) / int(capacity),4)

    stadium_row = stadiums_df.loc[stadiums_df['Team'] == home_team]

    if(stadium_row.empty):
        stadium_row = stadiums_df.loc[stadiums_df['Team'] == home_team + " women" if womens else " men"]
        if (stadium_row.empty):
            arena ="ERROR"
            city = "ERROR"
            state = "ERROR"
    else:
        stadium_row = stadium_row.iloc[0]
        arena = stadium_row['Arena']
        city = stadium_row['City']
        state = stadium_row['State']


    # add current row data to dictionary
    if atndnce != 0:
        data['Home_Team'].append(home_team)
        data["Attendance"].append(int(atndnce))
        data['Away_Team'].append(away_team)
        data["Date"].append(date)
        data["Capacity"].append(int(capacity))
        data['Percent_Capacity_Filled'].append(percent_cap)
        data["State"].append(state)
        data['City'].append(city)
        data['Arena'].append(arena)

schedule_df= pd.DataFrame(data)

In [10]:
schedule_df

,Home_Team,Away_Team,Attendance,Capacity,Percent_Capacity_Filled,Date,State,City,Arena
0,East Tennessee State,South Carolina,12779,18000,0.7099,"Monday, November 7, 2022",TN,Johnson City,Freedom Hall Civic Center
1,San Diego State,Stanford,3012,7233,0.4164,"Monday, November 7, 2022",CA,San Diego,Viejas Arena
2,Southern,Iowa,7417,15056,0.4926,"Monday, November 7, 2022",LA,Baton Rouge,F. G. Clark Center
3,Cincinnati,Louisville,8049,22000,0.3659,"Monday, November 7, 2022",OH,Cincinnati,Fifth Third Arena
4,Cleveland State,Iowa State,9556,14356,0.6656,"Monday, November 7, 2022",OH,Cleveland,Wolstein Center
...,...,...,...,...,...,...,...,...,...
152,Wisconsin,Milwaukee,2046,3500,0.5846,"Monday, November 7, 2022",WI,Madison,Kohl Center
153,Wisconsin,Milwaukee,2046,3500,0.5846,"Monday, November 7, 2022",WI,Madison,Kohl Center
154,New Orleans,UTEP,758,12000,0.0632,"Monday, November 7, 2022",LA,New Orleans,Lakefront Arena
155,Air Force,Denver,421,2400,0.1754,"Monday, November 7, 2022",CO,Colorado Springs,Cadet Field House
